# DRAC2022 Task1 Segmentation
## Data prepare

In [ ]:
import os
import shutil
from glob import glob
import cv2
import numpy as np
from tqdm import tqdm

# 设置图片和掩码的路径
image_dir = './inputs/DRAC-Segmentation/1. Original Images/a. Training Set'
mask_dir = './inputs/DRAC-Segmentation/2. Groundtruths/a. Training Set/3. Neovascularization'
output_dir = './inputs/DRAC-Segmentation/1. Original Images/3. Training Set'

# 确保输出目录存在
os.makedirs(output_dir, exist_ok=True)

# 读取掩码文件名
mask_files = glob(os.path.join(mask_dir, '*.png'))  # 假设掩码扩展名为.png
mask_basenames = {os.path.splitext(os.path.basename(mask))[0] for mask in mask_files}

# 遍历图片文件
image_files = glob(os.path.join(image_dir, '*.png'))  # 假设图片扩展名为.png
for image_path in image_files:
    image_basename = os.path.splitext(os.path.basename(image_path))[0]
    
    # 如果图片的基本名在掩码文件名集中，表示两者匹配
    if image_basename in mask_basenames:
        # 复制文件到输出目录
        shutil.copy(image_path, os.path.join(output_dir, os.path.basename(image_path)))
    else:
        pass

print("Filtering and copying complete.")

In [ ]:
img_size = 1024

image_dir = './inputs/DRAC-Segmentation/1. Original Images/3. Training Set'
mask_dir = './inputs/DRAC-Segmentation/2. Groundtruths/a. Training Set/3. Neovascularization'

os.makedirs('./inputs/DRAC2022/3.Neo/train/images', exist_ok=True)
os.makedirs('./inputs/DRAC2022/3.Neo/train/masks/0', exist_ok=True)

image_paths = glob(os.path.join(image_dir, '*'))

for imgpath in tqdm(image_paths):
    img = cv2.imread(imgpath)

    # 根据图像文件名找到对应的掩码文件
    base_name = os.path.splitext(os.path.basename(imgpath))[0]
    mask_path = glob(os.path.join(mask_dir, base_name + '.png'))  # 假设掩码文件名与图像文件名匹配

    if mask_path:  # 确保找到了掩码文件
        mask_path = mask_path[0]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask_ = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) > 127  # 读取并二值化掩码文件，生成布尔矩阵
        mask[mask_] = 1  # 将对应位置的零矩阵更新为1，表示掩码区域
        if len(img.shape) == 2:
            img = np.tile(img[..., None], (1, 1, 3))
        if img.shape[2] == 4:
            img = img[..., :3]

        img = cv2.resize(img, (img_size, img_size))
        mask = cv2.resize(mask, (img_size, img_size))

        cv2.imwrite(os.path.join('inputs/DRAC2022/3.Neo/train/images', os.path.basename(imgpath)), img)
        cv2.imwrite(os.path.join('inputs/DRAC2022/3.Neo/train/masks/0', base_name + '.png'), (mask * 255).astype('uint8'))
    else:
        print(f"No matching mask found for {imgpath}")

## Train Process

In [3]:
# !python train.py --name DRAC2022/3.Neo/train_NestedUNet_woDS --dataset DRAC2022/3.Neo/train -b 1 --epochs 500 --optimizer Adam --lr 1e-4 --min_lr 1e-9
# !python train-unet.py --arch UNet --name DRAC2022/3.Neo/train_NestedUNet_woDS --dataset DRAC2022/3.Neo/train -b 1 --optimizer Adam --epochs 500 --lr 4e-5 --min_lr 1e-9 --weights models/DRAC2022/3.Neo/train_UNet_woDS/Dice_model_epoch_160_loss_0.4863_dice_0.5816.pth
# !python train.py --dataset dsb2018_96 --optimizer Adam --min_lr 1e-9
!python train-unet.py --arch UNet --name DRAC2022/3.Neo/train_UNet_woDS --dataset DRAC2022/3.Neo/train -b 1 --optimizer Adam --epochs 200 --lr 4e-5 --min_lr 1e-9 --weights models/DRAC2022/3.Neo/train_UNet_woDS/Dice_model_epoch_160_loss_0.4863_dice_0.5816.pth

--------------------
name: DRAC2022/3.Neo/train_NestedUNet_woDS
epochs: 100
batch_size: 1
arch: NestedUNet
deep_supervision: False
input_channels: 3
num_classes: 1
input_w: 1024
input_h: 1024
loss: DiceLoss
dataset: DRAC2022/3.Neo/train
img_ext: .png
mask_ext: .png
optimizer: Adam
lr: 1e-05
momentum: 0.9
weight_decay: 0.0001
nesterov: False
scheduler: MultiStepLR
min_lr: 1e-09
factor: 0.1
patience: 2
milestones: 100
gamma: 0.1
early_stopping: -1
weights: models/DRAC2022/3.Neo/train_NestedUNet_woDS/Dice_model_epoch_100_loss_0.3334_dice_0.6677.pth
num_workers: 4
--------------------
=> creating model NestedUNet
=> loaded weights from 'models/DRAC2022/3.Neo/train_NestedUNet_woDS/Dice_model_epoch_100_loss_0.3334_dice_0.6677.pth'
Epoch [0/100]
  0%|                                                    | 0/28 [00:00<?, ?it/s]/home/zhouxvdong/.conda/envs/neunet/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered

## Validation

In [ ]:
!python val.py --name DRAC2022/3.Neo/train_NestedUNet_woDS

## Prediction

In [ ]:
import os
from glob import glob
import cv2
import numpy as np
from tqdm import tqdm

img_size = 1024

image_dir = './inputs/DRAC-Segmentation/1. Original Images/b. Testing Set'

os.makedirs('./inputs/DRAC2022/3.Neo/test/images', exist_ok=True)

image_paths = glob(os.path.join(image_dir, '*'))

for imgpath in tqdm(image_paths):
    img = cv2.imread(imgpath)

    if len(img.shape) == 2:
        img = np.tile(img[..., None], (1, 1, 3))
    if img.shape[2] == 4:
        img = img[..., :3]

    img = cv2.resize(img, (img_size, img_size))
    cv2.imwrite(os.path.join('inputs/DRAC2022/3.Neo/test/images', os.path.basename(imgpath)), img)

In [1]:
!python predict3.py --name DRAC2022/3.Neo/train_NestedUNet_woDS

--------------------
arch: NestedUNet
batch_size: 1
dataset: DRAC2022/3.Neo/train
deep_supervision: False
early_stopping: -1
epochs: 100
factor: 0.1
gamma: 0.1
img_ext: .png
input_channels: 3
input_h: 1024
input_w: 1024
loss: DiceLoss
lr: 1e-05
mask_ext: .png
milestones: 100
min_lr: 1e-09
momentum: 0.9
name: DRAC2022/3.Neo/train_NestedUNet_woDS
nesterov: False
num_classes: 1
num_workers: 4
optimizer: Adam
patience: 2
scheduler: MultiStepLR
weight_decay: 0.0001
weights: models/DRAC2022/3.Neo/train_NestedUNet_woDS/Dice_model_epoch_100_loss_0.3334_dice_0.6677.pth
--------------------
=> creating model NestedUNet
  0%|                                                    | 0/65 [00:00<?, ?it/s]/home/zhouxvdong/.conda/envs/neunet/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|█████████████